<center>
<h3> </h3> <!--Por alguna razon, esto es lo que permite que se centre-->
<h1>
  Proyecto: <b><u>World University Rankings(2024-5)</b></u> 
</h1>
<h3>
  Laboratorio 1
</h3>

---

<h4>
  <b> Grupo 4: </b>  &nbsp; Vicente Rodriguez Rogers &nbsp; - &nbsp; Cristian Rioseco &nbsp; - &nbsp; Isidora Lemunao&nbsp;  -  &nbsp;Martin XXXXXXXX 
</h4>
</center>

***

<div style="text-align: center;">
  <img src="imagenes/picture_1.png" alt="Texto alternativo" />
</div>

### **Motivacion**
#### &nbsp; &nbsp; &nbsp; Como estudiantes universitarios la calidad y el poscionamiento de las instituciones de educación superior son temas de sumo interés, Es por esto y por el valor intrinsico del analisis del ranking de universidades que decidicimos realizar nuestro proyecto sobre este tema.

#### Nos sentimos particularmente motivados a explorar la dispersión de la calidad educativa a un nivel global. Entender cómo se distribuyen los centros de excelencia académica puede ser un conocimiento crucial a la hora de entender el como y donde se generan instituciones mas capacitadas para otorgar una educación de mayor valor.

#### Este análisis nos permitirá no solo comprender mejor el panorama universitario global, sino también extraer conclusiones valiosas sobre los atributos que definen el éxito y la reputación de una institución.

<center>
    <h2> <b><u> Análisis exploratorio de datos</b></u>
</center>

### **Limpieza de Datos**

En esta sección se hará la limpieza del dataset con el objetivo de dejarlo en un estado utilizable para el completo análisis de nuestro interés. Se aplicaron ciertas transformaciones, en las cuales están:


*   Eliminación de filas vacías, duplicadas o que correspondan a encabezados repetidos.
*   Normalización de nombres de columnas para evitar duplicados y facilitar su lectura.
*   Conversión de variables a su tipo de dato correcto.
*   Estandarización de valores.

Como resultado, se obtiene un conjunto de datos limpio con 1503 instituciones registradas, de las cuales 600 poseen un puntaje global (OverallScore) disponible para los análisis posteriores.

In [ ]:
import pandas as pd
df = pd.read_excel("Dataset/2025 QS World University Rankings 2.2 (For qs.com).xlsx", header=2)
df.head()

,INDEX,RANK,RANK.1,Unnamed: 3,Location,Region,SIZE,FOCUS,RES.,STATUS,SCORE,RANK.2,SCORE.1,RANK.3,SCORE.2,RANK.4,SCORE.3,RANK.5,SCORE.4,RANK.6,SCORE.5,RANK.7,SCORE.6,RANK.8,SCORE.7,RANK.9,Score,Rank,SCORE.8
0,INDEX,rank display,rank display2,institution,location code,location,size,focus,research,status,ar score,ar rank,er score,er rank,fsr score,fsr rank,cpf score,cpf rank,ifr score,ifr rank,isr score,isr rank,irn score,irn rank,ger score,ger rank,SUS SCORE,SUS RANK,Overall Score
1,1,1,1,Massachusetts Institute of Technology (MIT),United States,Americas,M,CO,VH,B,100,4,100,2,100,11,100,9,99.3,100,86.8,143,96,58,100,8,99,15=,100
2,2,2,6,Imperial College London,United Kingdom,Europe,L,FC,VH,A,98.5,22,99.5,11,98.2,45,93.9,54,100,66,99.6,44,97.4,34,93.4,61,99.7,6,98.5
3,3,3,3,University of Oxford,United Kingdom,Europe,L,FC,VH,A,100,2,100,5,100,8,84.8,93,98.1,120,97.7,73,100,1,100,3,85,126,96.9
4,4,4,4,Harvard University,United States,Americas,L,FC,VH,B,100,1,100,1,96.3,53,100,1,74.1,269,69,215,99.6,5,100,1,84.4,130,96.8


En el extracto anterior se visualizan las primeras filas tal como se cargaron, para verificar estructura y encabezados.

In [13]:
import pandas as pd, numpy as np, re

fn = "Dataset/2025 QS World University Rankings 2.2 (For qs.com).xlsx"

def make_unique(cols):
    seen, out = {}, []
    for c in cols:
        if c in seen: seen[c]+=1; out.append(f"{c}.{seen[c]}")
        else: seen[c]=0; out.append(c)
    return out

def find_col(df, options):
    low = {c.lower(): c for c in df.columns}
    for opt in options:
        if opt in low: return low[opt]
    for c in df.columns:
        cl = c.lower()
        if any(opt in cl for opt in options): return c
    return None

def clean_rank(x):
    if pd.isna(x): return np.nan
    s = str(x).replace("=", "")
    s = re.split(r"[-–]", s)[0]
    s = re.sub(r"[^\d]","", s)
    return pd.to_numeric(s, errors="coerce")

df = pd.read_excel(fn, header=1)
if isinstance(df.iloc[0,0], str) and df.iloc[0,0].strip().upper()=="INDEX":
    df = df.drop(0).reset_index(drop=True)

df.columns = make_unique(df.columns)
df.columns = df.columns.map(str)
df = df.dropna(how="all").drop_duplicates()
obj = df.select_dtypes(include="object").columns
df[obj] = df[obj].apply(lambda s: s.astype(str).str.strip())

m = {}
c = find_col(df, ["institution name","institution"]);         m[c]="Institution"            if c else None
c = find_col(df, ["country","location"]);                     m[c]="Country"                if c else None
c = find_col(df, ["overall score","overall"]);                m[c]="OverallScore"           if c else None
c = find_col(df, ["sustainabilityscore","sustainability score","sus score"]);  m[c]="SustainabilityScore" if c else None
c = find_col(df, ["sustainabilityrank","sustainability rank","sus rank"]);     m[c]="SustainabilityRank"  if c else None
c = find_col(df, ["rank2025","rank display"]);                m[c]="Rank2025"               if c else None
c = find_col(df, ["rank2024","rank display2"]);               m[c]="Rank2024"               if c else None
df = df.rename(columns={k:v for k,v in m.items() if k})

rank_cols  = [c for c in df.columns if str(c).upper().startswith("RANK")]
score_cols = [c for c in df.columns if str(c).upper().startswith("SCORE")] + ["OverallScore","SustainabilityScore"]

for c in rank_cols: df[c] = df[c].apply(clean_rank)
for c in score_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c].astype(str).str.replace("%","",regex=False).str.replace(",",".",regex=False), errors="coerce")

for c in ["Country","Region","SIZE","FOCUS","RES.","STATUS"]:
    if c in df.columns: df[c] = df[c].replace({"USA":"United States"}).astype("category")

if {"Institution","Country"}.issubset(df.columns):
    df = df[~df["Institution"].str.lower().eq("institution")]
    df = df[~df["Country"].astype(str).str.lower().eq("location code")]
    df = df.reset_index(drop=True)

keep = ["Institution","Country","Region","SIZE","FOCUS","RES.","STATUS","Rank2025","Rank2024","OverallScore","SustainabilityScore","SustainabilityRank"]
df_clean = df[[c for c in keep if c in df.columns]].copy()
df_scores = df_clean.dropna(subset=["OverallScore"]).reset_index(drop=True)

df_clean.to_csv("qs_rankings_clean.csv", index=False)
df_scores.to_csv("qs_rankings_clean_with_scores.csv", index=False)

print(df_clean.shape, df_scores.shape)


(1503, 3) (600, 3)


### **Descripción de variables y datos**

 En este apartado nos encargaremos de realizar un análisis sobre distintos aspectos de nuestra base de datos, es por ello que para facilitar todo esto, haremos uso de la libreria "Skimpy", asi que partiremos con la instalación de este en caso de no poseerlo;

In [ ]:
pip install skimpy

Ahora procedemos a generar una tabla con un resumen de los datos encontrados en nuestro dataset.

In [ ]:
#----------Settings----------
import pandas as pd
from  skimpy import skim

#-----Settings visuales------
#modificamos el ancho y cantidad de columnas que se muestran
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_colwidth', None)

#----------Carga-------------
dataset_rankings = pd.read_csv('qs_rankings_clean_with_scores.csv')

skim(dataset_rankings)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ Dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 600    │ │ string      │ 2     │                                                          │
│ │ Number of columns │ 3      │ │ float64     │ 1     │                                                          │
│ └───────────────────┴────────┘ └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━┳━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━━┓  │
│ ┃ column           ┃ NA  ┃ NA %   ┃ mean    ┃ sd      ┃ p0     ┃ p25     ┃ p50     ┃ p75   ┃ p100  ┃ hist    ┃  │
│ ┡━━━━━━━━━━━━━━━━━━╇━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━━┩  │
│ │ OverallScore     │   0 │      0 │   41.84 │   18.83 │   20.8 │   26.58 │   36.35 │  52.1 │   100 │ █▄▃▁▁▁  │  │
│ └──────────────────┴─────┴────────┴─────────┴─────────┴────────┴─────────┴─────────┴───────┴───────┴─────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━┳━━━━┳━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┓  │
│ ┃           ┃    ┃      ┃          ┃           ┃           ┃           ┃ chars per  ┃ words per ┃ total      ┃  │
│ ┃ column    ┃ NA ┃ NA % ┃ shortest ┃ longest   ┃ min       ┃ max       ┃ row        ┃ row       ┃ words      ┃  │
│ ┡━━━━━━━━━━━╇━━━━╇━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━┩  │
│ │ Instituti │  0 │    0 │ UCL      │ Tashkent  │ Aalborg   │ École     │       27.6 │       3.6 │       2140 │  │
│ │ on        │    │      │          │ Institute │ Universit │ Normale   │            │           │            │  │
│ │           │    │      │          │ of        │ y         │ Supérieur │            │           │            │  │
│ │           │    │      │          │ Irrigatio │           │ e de Lyon │            │           │            │  │
│ │           │    │      │          │ n and     │           │           │            │           │            │  │
│ │           │    │      │          │ Agricultu │           │           │            │           │            │  │
│ │           │    │      │          │ ral       │           │           │            │           │            │  │
│ │           │    │      │          │ Mechaniza │           │           │            │           │            │  │
│ │           │    │      │          │ tion      │           │           │            │           │            │  │
│ │           │    │      │          │ Engineers │           │           │            │           │            │  │
│ │           │    │      │          │ -         │           │           │            │           │            │  │
│ │           │    │      │          │ National  │           │           │            │           │            │  │
│ │           │    │      │          │ Research  │           │           │            │           │            │  │
│ │           │    │      │          │ Universit │           │           │            │           │            │  │
│ │           │    │      │          │ y         │           │           │            │           │            │  │
│ │           │    │      │          │ (TIIAME-N │      

In [7]:
#----------Settings----------
import pandas as pd
from  skimpy import skim

#-----Settings visuales------
#modificamos el ancho y cantidad de columnas que se muestran
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_colwidth', None)

#----------Carga-------------
dataset_rankings = pd.read_csv('Dataset/World University Ranking 2025.csv',
                                header = 2,            # Usamos la tercera linea como encabezado
                                decimal = ',',         
                                thousands = '.'        
                                )
#ahora nos encargamos de eliminar la 4 linea
dataset_rankings = dataset_rankings.iloc[1:].copy()
#ahora reiniciamos el indice
dataset_rankings = dataset_rankings.reset_index(drop=True)

skim(dataset_rankings)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ Dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 1503   │ │ string      │ 19    │                                                          │
│ │ Number of columns │ 29     │ │ float64     │ 10    │                                                          │
│ └───────────────────┴────────┘ └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━┳━━━━━━━┳━━━━━━┳━━━━━━┳━━━━━━┳━━━━━━━━┓  │
│ ┃ column              ┃ NA  ┃ NA %               ┃ mean  ┃ sd    ┃ p0  ┃ p25   ┃ p50  ┃ p75  ┃ p100 ┃ hist   ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━╇━━━━━━━╇━━━━━━╇━━━━━━╇━━━━━━╇━━━━━━━━┩  │
│ │ INDEX               │   0 │                  0 │   752 │   434 │   1 │ 376.5 │  752 │ 1128 │ 1503 │ ██████ │  │
│ │ Academic reputation │   0 │                  0 │ 20.29 │ 22.33 │ 1.3 │   6.3 │   11 │ 23.5 │  100 │  █▂▁   │  │
│ │ Score               │     │                    │       │       │     │       │      │      │      │        │  │
│ │ Employer Reputation │   0 │                  0 │ 19.79 │ 23.78 │ 1.1 │   4.3 │  9.4 │ 25.3 │  100 │ █▂▁  ▁ │  │
│ │ score               │     │                    │       │       │     │       │      │      │      │        │  │
│ │ Faculty Student     │   0 │                  0 │ 28.13 │ 27.61 │   1 │   7.3 │   16 │ 40.4 │  100 │ █▃▂▁▁▁ │  │
│ │ Score               │     │                    │       │       │     │       │      │      │      │        │  │
│ │ Citations per       │   0 │                  0 │  23.5 │ 27.87 │   1 │   2.8 │  9.9 │ 36.4 │  100 │ █▂▁▁▁▁ │  │
│ │ Faculty score       │     │                    │       │       │     │       │      │      │      │        │  │
│ │ International       │ 100 │   6.65335994677312 │ 30.74 │ 34.34 │   1 │   4.2 │ 12.5 │ 51.6 │  100 │ █▂▁▁▁▂ │  │
│ │ Faculty score       │     │                    │       │       │     │       │      │      │      │        │  │
│ │ International       │  58 │ 3.8589487691284097 │ 25.58 │  31.1 │   1 │   2.9 │  9.6 │ 38.3 │  100 │ █▂▁▁▁▁ │  │
│ │ Students Score      │     │                    │       │       │     │       │      │      │      │        │  │
│ │ International       │   1 │ 0.0665335994677312 │ 50.13 │ 29.87 │   1 │ 22.73 │ 51.1 │ 77.1 │  100 │ █▆▅▆▇▇ │  │
│ │ Research Network    │     │                    │       │       │     │       │      │      │      │        │  │
│ │ SCORE               │     │                    │       │       │     │       │      │      │      │        │  │
│ │ Employment Outcomes │   0 │                  0 │ 23.83 │ 27.35 │ 1.2 │     4 │ 11.8 │ 33.4 │  100 │ █▂▁▁▁▁ │  │
│ │ Socre               │     │                    │       │       │     │       │      │      │      │        │  │
│ │ Sustainability      │  19 │  1.264138389886893 │ 24.31 │ 31.07 │   1 │   1.3 │ 6.75 │ 39.5 │  100 │ █▁▁▁▁▁ │  │
│ │ Score               │     │                    │       │       │     │       │      │      │      │        │  │
│ └─────────────────────┴─────┴────────────────────┴───────┴───────┴─────┴───────┴──────┴──────┴──────┴────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━┳━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━

In [16]:
#----------Settings----------
import pandas as pd
from  skimpy import skim

#-----Settings visuales------
#modificamos el ancho y cantidad de columnas que se muestran
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_colwidth', None)

#----------Carga-------------
dataset_rankings = pd.read_csv('qs_rankings_clean.csv')

skim(dataset_rankings)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ Dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 1503   │ │ string      │ 2     │                                                          │
│ │ Number of columns │ 3      │ │ float64     │ 1     │                                                          │
│ └───────────────────┴────────┘ └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━┳━━━━━━┳━━━━━━━━┓  │
│ ┃ column        ┃ NA   ┃ NA %                ┃ mean   ┃ sd     ┃ p0   ┃ p25   ┃ p50   ┃ p75  ┃ p100 ┃ hist   ┃  │
│ ┡━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━╇━━━━━━╇━━━━━━━━┩  │
│ │ OverallScore  │  903 │  60.079840319361274 │  41.84 │  18.83 │ 20.8 │ 26.58 │ 36.35 │ 52.1 │  100 │ █▄▃▁▁▁ │  │
│ └───────────────┴──────┴─────────────────────┴────────┴────────┴──────┴───────┴───────┴──────┴──────┴────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━┳━━━━┳━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┓  │
│ ┃           ┃    ┃      ┃          ┃           ┃           ┃           ┃ chars per  ┃ words per ┃ total      ┃  │
│ ┃ column    ┃ NA ┃ NA % ┃ shortest ┃ longest   ┃ min       ┃ max       ┃ row        ┃ row       ┃ words      ┃  │
│ ┡━━━━━━━━━━━╇━━━━╇━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━┩  │
│ │ Instituti │  0 │    0 │ UCL      │ Tashkent  │ AGH       │ Óbuda     │       29.2 │       3.8 │       5684 │  │
│ │ on        │    │      │          │ Institute │ Universit │ Universit │            │           │            │  │
│ │           │    │      │          │ of        │ y of      │ y         │            │           │            │  │
│ │           │    │      │          │ Irrigatio │ Science   │           │            │           │            │  │
│ │           │    │      │          │ n and     │ and       │           │            │           │            │  │
│ │           │    │      │          │ Agricultu │ Technolog │           │            │           │            │  │
│ │           │    │      │          │ ral       │ y         │           │            │           │            │  │
│ │           │    │      │          │ Mechaniza │           │           │            │           │            │  │
│ │           │    │      │          │ tion      │           │           │            │           │            │  │
│ │           │    │      │          │ Engineers │           │           │            │           │            │  │
│ │           │    │      │          │ -         │           │           │            │           │            │  │
│ │           │    │      │          │ National  │           │           │            │           │            │  │
│ │           │    │      │          │ Research  │           │           │            │           │            │  │
│ │           │    │      │          │ Universit │           │           │            │           │            │  │
│ │           │    │      │          │ y         │           │           │            │           │            │  │
│ │           │    │      │          │ (TIIAME-N │      

 ***Observaciones:*** &nbsp; El resultado que nos entrego Skimpy nos permite sacar varios 